In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pdb
import math

torch.cuda.empty_cache()

def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop, x, y

def searchForFocus(filename, substring):
    with open(filename, 'r') as file:
        data = file.read()
        location = data.find(substring)
        croppedStr = data[location+len(substring):]
        # Split at spaces and find first number
        for word in croppedStr.split(): # Split at spaces
            # Delete any commas    
            word = word.replace(',', "")
            try:
                focusPosition = int(word)
                return focusPosition
            except ValueError:
                continue
    file.close()

class Dataset(torch.utils.data.Dataset):
    # ids indicates what subfolders (samples) to access
    def __init__(self, foldername, subfolderPrefix, ids):
        self.foldername = foldername
        self.subfolderPrefix = subfolderPrefix
        self.ids = ids
        
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        sampleFoldername = self.foldername + '/' + self.subfolderPrefix + str(index)
        
        # H, W
        cropSize = (640, 640)
        
        images = []
        for i, prefix in enumerate(['before', 'after']):
        
            # Load in image as [0,1] array
            image = cv2.imread(sampleFoldername + '/' + prefix + str(index) + '.tif', 0) * 1 / 255.0

            # Shift it so is from [-1,1]
            image *= 2
            image -= 1
            
            if i == 0:
                # Randomly crop the image
                image, cornerX, cornerY = get_random_crop(image, cropSize[0], cropSize[1])
            else:
                # Crop the label image to the same region as the input
                image = image[cornerY:cornerY + cropSize[0], cornerX:cornerX + cropSize[1]]
            
            temp = torch.from_numpy(image)
            if i == 0:
                temp = temp.unsqueeze(0) # Add fake first dimension to specify 1-channel
            images.append(temp)
            
        return images
    

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

params = {'batch_size': 2,
          'shuffle': True,
          'num_workers': 2}

# Randomly partition the full list into a training set and validation set
numSamples = 100 # total number of samples collected
frac = 1/5 # fraction to be validation
np.random.seed(0)
permutedIds = np.random.permutation(range(numSamples))
splitPoint = int((1-frac) * len(permutedIds))
trainingIds = permutedIds[:splitPoint]
valIds = permutedIds[splitPoint:]

training_set = Dataset('/home/aofeldman/Desktop/AFdataCollection', 'sample', trainingIds)
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = Dataset('/home/aofeldman/Desktop/AFdataCollection', 'sample', valIds)
#validation_generator = torch.utils.data.DataLoader(validation_set, **params)

In [29]:
X, Y = training_set.__getitem__(1)

def imshow(img,wait):
    img = img.squeeze()
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    print(npimg)
    #width = int(0.15 * npimg.shape[1])
    #height = int(0.15 * npimg.shape[0])
    #cv2.imshow("Hi",cv2.resize(npimg, (width, height)))
    cv2.imshow('hi', npimg)
    cv2.waitKey(wait)
    cv2.destroyAllWindows()
imshow(X, 10000)
print(X.shape)
imshow(Y, 10000)
print(Y.shape)

[[0.2        0.20392157 0.18823529 ... 0.45098039 0.45882353 0.44705882]
 [0.18823529 0.19215686 0.19607843 ... 0.41568627 0.42352941 0.41960784]
 [0.19215686 0.17647059 0.18823529 ... 0.36862745 0.37254902 0.39215686]
 ...
 [0.18823529 0.19215686 0.20784314 ... 0.03529412 0.05882353 0.03921569]
 [0.21176471 0.19607843 0.21176471 ... 0.01176471 0.03921569 0.04313725]
 [0.22352941 0.21568627 0.21568627 ... 0.05098039 0.05882353 0.0627451 ]]
torch.Size([1, 640, 640])
[[0.18431373 0.20784314 0.2        ... 0.04313725 0.07058824 0.07058824]
 [0.19607843 0.21568627 0.20392157 ... 0.04313725 0.05882353 0.05882353]
 [0.19215686 0.16862745 0.16470588 ... 0.04313725 0.05490196 0.0745098 ]
 ...
 [0.19215686 0.19607843 0.20392157 ... 0.04313725 0.05882353 0.03529412]
 [0.20392157 0.2        0.20392157 ... 0.01176471 0.03137255 0.05098039]
 [0.21176471 0.21568627 0.20784314 ... 0.02745098 0.05098039 0.0627451 ]]
torch.Size([640, 640])


In [7]:
# Consider placing dropout layers after conv2d layers (conv2d -> batchnorm2d -> leakyReLU -> dropout(p=0.1))
# And also place after fully connected layers (linear -> leakyReLU -> dropout(p=0.3))
# TODO: Should actually figure out appropriate amount of padding for layers 

net = nn.Sequential(
    # Encoder section
    
    # Does not change channel dimensions
    nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    # 1/4 channel dimensions
    nn.MaxPool2d(kernel_size=4, stride=4),
    # Does not change channel dimensions
    nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    # 1/4 channel dimensions
    nn.MaxPool2d(kernel_size=4, stride=4),
    # Does not change channel dimensions
    nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    # 1/4 channel dimensions
    nn.MaxPool2d(kernel_size=4, stride=4),
    
    # At this point:
    # Each channel has dimensions
    # H_out, W_out = (1/4)^3 * (H_in, W_in)
    
    # Decoder section
    nn.ConvTranspose2d(4, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    nn.Upsample(scale_factor = 4, mode='bilinear'),
    
    nn.ConvTranspose2d(4, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    nn.Upsample(scale_factor = 4, mode='bilinear'),
    
    nn.ConvTranspose2d(4, 4, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(4),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    nn.Upsample(scale_factor = 4, mode='bilinear'),
    
#     nn.ConvTranspose2d(4, 4, kernel_size=3, stride=1, padding=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.Upsample(scale_factor = (2, 2)),
    
    nn.ConvTranspose2d(4, 1, kernel_size=3, stride=1, padding=1),
    nn.Tanh(),
)

net = net.to(device)

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, dim, kernel, leakySlope, poolSize, use_norm):
        block = [nn.Conv2d(dim[0], dim[1], kernel_size=kernel, stride=1, padding= (kernel-1) // 2)]
        
        if use_norm:
            block += [nn.BatchNorm2d(dim[1])]
        block += [nn.LeakyReLU(negative_slope = leakySlope, inplace=True),
                  nn.MaxPool2d(kernel_size=poolSize, stride=poolSize)]
        
        self.encoderBlock = nn.Sequential(*block)
        
    def forward(self, x):
        return self.decoderBlock(x)
    
class DecoderBlock(nn.Module):
    def __init__(self, dim, kernel, leakySlope, scale, use_norm):
        block = [nn.ConvTransposed2d(dim[0], dim[1], kernel_size=kernel, stride=1, padding= (kernel-1) // 2)]
        
        if use_norm:
            block += [nn.BatchNorm2d(dim[1])]
        block += [nn.LeakyReLU(negative_slope = leakySlope, inplace=True),
                  nn.Upsample(scale_factor = scale, mode = 'bilinear')]
        
        self.decoderBlock = nn.Sequential(*block)
    
    def forward(self, x):
        return self.decoderBlock(x)
    
class Net(nn.Module):
    def __init__(self, numEncoder, numDecoder):
        super(Net, self).__init__()
        
        self.model = nn.Sequential([])
        for i in range(numEncoder):
            self.model = nn.Sequential(self.model, EncoderBlock(1, 4, 5, 0.1, 4, True)
        for j in range(numDecoder):
            self.model = nn.Sequential(self.model, DecoderBlock(1, 4, 5, 0.1, 4, True)
    
    def forward(self, x):
        return self.model(x)
                                       
net = Net().to(device)

In [8]:
for p in net.parameters():
    print(p.data.shape)

torch.Size([4, 1, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 4, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 4, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 4, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 4, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 4, 5, 5])
torch.Size([4])
torch.Size([4])
torch.Size([4])
torch.Size([4, 1, 3, 3])
torch.Size([1])


In [9]:
count = 0
for p in net.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
    print(p.data)
print(f'total params: {count}')

tensor([[[[ 0.0896, -0.1148, -0.1917, -0.0993, -0.0427],
          [ 0.1118,  0.1566,  0.1642, -0.1485, -0.1511],
          [-0.0448, -0.0137, -0.1080,  0.0233, -0.1919],
          [ 0.1708, -0.0452,  0.0264, -0.1029,  0.1297],
          [-0.0015, -0.0838,  0.1368, -0.1938,  0.1623]]],


        [[[ 0.1647, -0.0594,  0.0275, -0.1991,  0.0152],
          [ 0.1332,  0.0326,  0.1105, -0.1027,  0.0334],
          [-0.0466,  0.0208, -0.0144,  0.1781,  0.0878],
          [-0.1738,  0.0425, -0.0143, -0.1736, -0.0191],
          [ 0.0367, -0.0522, -0.0127,  0.0433,  0.1296]]],


        [[[-0.0949, -0.1031,  0.0034, -0.1936, -0.1275],
          [ 0.0719, -0.1426, -0.0033,  0.0595, -0.1780],
          [-0.0260,  0.0856, -0.1913, -0.1365,  0.0193],
          [ 0.0905, -0.1401,  0.1196, -0.1332,  0.1922],
          [-0.0111, -0.1348,  0.0926,  0.0061,  0.0548]]],


        [[[ 0.0282,  0.0165,  0.1225, -0.1845,  0.0910],
          [ 0.1853,  0.1841,  0.0828, -0.1426,  0.1555],
          [-0.1128,

In [10]:
import torch.optim as optim

criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.RMSprop(net.parameters())

In [11]:
# For early stopping, use blocks of epochs (say size 3), and a window (say size 10) of how far to look in the past.
# Then, compare the average loss on the current block to the block in the past. Another idea would be compare the variances.
# If the fraction is not sufficiently small, halt.

max_epochs = 100
# Should be stated as a fraction of the previous error
#max_frac = 0.999
#window = 10 # How many epochs in the past to compare to
#block = 5 # What size block of epochs to use

learnFreq = 10
batch_multiplier = 1

epoch_training_loss = []
epoch_val_loss = []


for epoch in range(max_epochs):
    print("\nOn epoch: " + str(epoch))

    count = 0
    
    net.train()
    for inputs, labels in training_generator:
        
        inputs, labels = inputs.to(device).float(), labels.to(device).float()
        
        if count == 0:
            optimizer.step()
            # zero the parameter gradients
            optimizer.zero_grad()
            count = batch_multiplier

        # forward + backward + optimize
        outputs = net(inputs).to(device)
        loss = criterion(torch.squeeze(outputs), labels) / batch_multiplier
        loss.backward()
        #optimizer.step()
        
        count -=1
        
        loss = loss.detach()
        inputs = inputs.detach()
        outputs = outputs.detach()
        
        #print('Outputs', torch.squeeze(outputs))
        #print('Batch size: ' + str(len(inputs)))
        # Multiply by the batch size and batch_multiplier (because earlier divided)
        #running_loss += loss.item() * len(inputs) * batch_multiplier
        #print('Batch average loss ' + str(loss.item()))
    #training_loss = running_loss / training_set.__len__()
    #print("Epoch training loss: " + str(training_loss))
    #epoch_training_loss.append(training_loss)
    if epoch % learnFreq == 0 or epoch == (max_epochs - 1):
        with torch.no_grad():
            net.eval()
            for i, dataset in enumerate([training_set, validation_set]):
                if i == 0:
                    print('\nEpoch training results ')
                else:
                    print('\nEpoch validation results ')
                MSE = 0
                avgAbsDev = 0
                for sample in dataset.ids:
                    X, y = dataset.__getitem__(sample)
                    X = X.unsqueeze(0) # Add fake batch dimension
                    X = X.to(device).float()
        
                    yHat = net(X).to(device)
                    yHat = yHat.to('cpu')
                    MSE += torch.norm(y - yHat)**2 / np.product(y.numpy().shape)
                    #avgAbsDev += np.abs(yHat - y)
        
                #print('Sample: ' + str(sample))
                #print('y: ' + str(y))
                #print('yHat: ' + str(yHat))
                MSE /= len(dataset.ids)
                #avgAbsDev /= len(dataset.ids)
                print('RMSE on dataset: ' + str(np.sqrt(MSE)))
                #print('Avg Abs Dev on dataset: ' + str(avgAbsDev))
                if i == 0:
                    epoch_training_loss.append(MSE)
                else:
                    epoch_val_loss.append(MSE)
        
# for epoch in range(max_epochs):
#     print("\nOn epoch: " + str(epoch))
        
#     net.train()
#     for inputs, labels in training_generator:
#         # zero the parameter gradients
#         optimizer.zero_grad()
#         print('inputs.shape: ', inputs.size())
        
#         # forward + backward + optimize
#         outputs = net(inputs.float())
#         print('outputs.shape: ', outputs.size())
#         print('labels.shape: ', labels.float().size())
#         loss = criterion(torch.squeeze(outputs), labels.float())
#         loss.backward()
#         optimizer.step()
#         print('Batch Loss: ' + str(loss.item()))
        
#     with torch.no_grad():
#         val_loss = 0
#         net.eval()
#         for ind in validation_set.ids:
#             X, y = validation_set.__getitem__(ind)
#             X = X.unsqueeze(0) # Add fake batch dimension
#             yHat = net(X.float())
#             val_loss += torch.norm(y - yHat)**2 / np.product(y.numpy().shape)
#         val_loss /= validation_set.__len__()
#         epoch_val_loss.append(val_loss)
#     print("Epoch validation loss: " + str(val_loss))
        
#     if len(epoch_val_loss) >= window + block + 1:
#         latestBlock = np.mean(epoch_val_loss[-1:-1-block])
#         earlierBlock = np.mean(epoch_val_loss[-1-window:-1-window-block])
        
#         # latestBlock must be sufficiently smaller than earlierBlock
#         if latestBlock / earlierBlock > max_frac:
#             print('Converged')
#             pdb.set_trace()
#             break
            
print('Finished Training')


On epoch: 0


/home/aofeldman/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3118: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "



Epoch training results 
RMSE on dataset: tensor(0.6046, dtype=torch.float64)

Epoch validation results 
RMSE on dataset: tensor(0.5912, dtype=torch.float64)

On epoch: 1

On epoch: 2

On epoch: 3

On epoch: 4

On epoch: 5

On epoch: 6

On epoch: 7

On epoch: 8

On epoch: 9

On epoch: 10

Epoch training results 
RMSE on dataset: tensor(0.5820, dtype=torch.float64)

Epoch validation results 
RMSE on dataset: tensor(0.5966, dtype=torch.float64)

On epoch: 11

On epoch: 12

On epoch: 13

On epoch: 14

On epoch: 15

On epoch: 16

On epoch: 17

On epoch: 18

On epoch: 19

On epoch: 20

Epoch training results 
RMSE on dataset: tensor(0.5777, dtype=torch.float64)

Epoch validation results 
RMSE on dataset: tensor(0.5927, dtype=torch.float64)

On epoch: 21

On epoch: 22

On epoch: 23

On epoch: 24

On epoch: 25

On epoch: 26

On epoch: 27

On epoch: 28

On epoch: 29

On epoch: 30

Epoch training results 
RMSE on dataset: tensor(0.5677, dtype=torch.float64)

Epoch validation results 
RMSE on da

KeyboardInterrupt: 

In [30]:
with torch.no_grad():
    net.eval()
    for i, dataset in enumerate([training_set, validation_set]):
        if i == 0:
            print('\nTraining results ')
        else:
            print('\nValidation results ')
        MSE = 0
        for sample in dataset.ids:
            X, y = dataset.__getitem__(sample)
            
            X = X.unsqueeze(0).to(device) # Add fake batch dimension
            yHat = net(X.float())
            
            yHat = yHat.to('cpu')
            
            imshow(X.to('cpu'), 10000)
            imshow(y, 10000)
            imshow(yHat, 10000)
            # squared frobenius norm
            MSE += torch.norm(y - yHat)**2 / np.product(y.numpy().shape)
        MSE /= validation_set.__len__()
    print('RMSE on dataset: ' + str(np.sqrt(MSE)))


Training results 
[[0.58823529 0.59215686 0.58431373 ... 0.55686275 0.55686275 0.55686275]
 [0.58431373 0.58823529 0.58431373 ... 0.55686275 0.55686275 0.55686275]
 [0.57647059 0.58431373 0.58431373 ... 0.56078431 0.56078431 0.56078431]
 ...
 [0.52941176 0.53333333 0.52941176 ... 0.41568627 0.43529412 0.43921569]
 [0.51764706 0.5254902  0.52156863 ... 0.41960784 0.43921569 0.43921569]
 [0.51764706 0.52941176 0.52941176 ... 0.42745098 0.43921569 0.43921569]]
[[0.10588235 0.06666667 0.05490196 ... 0.06666667 0.05098039 0.05490196]
 [0.10588235 0.07843137 0.05490196 ... 0.07843137 0.08235294 0.07843137]
 [0.07843137 0.10588235 0.08627451 ... 0.08235294 0.09411765 0.0627451 ]
 ...
 [0.16862745 0.21568627 0.37647059 ... 0.18431373 0.2        0.18823529]
 [0.12941176 0.20392157 0.36862745 ... 0.1254902  0.13333333 0.11372549]
 [0.1372549  0.20392157 0.33333333 ... 0.12156863 0.14509804 0.12156863]]
[[0.33435106 0.34369275 0.3443289  ... 0.33058786 0.33043936 0.3297859 ]
 [0.3473851  0.36539

KeyboardInterrupt: 

In [ ]:
%matplotlib qt5
plt.figure()
plt.plot(range(max_epochs), epoch_training_loss, range(max_epochs), epoch_val_loss)
plt.legend(['Training', 'Validation'])